In [20]:
# 작성자 : 이민우 
# 작성일 20181125
# 프로그램 설명 : 부스팅 예제 

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons # moons 데이터셋 사용 
from sklearn.ensemble import AdaBoostClassifier 
#from sklearn.tree import DecisionTreeClassifier # 결정 트리 
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor # GBRT 간단하게 가능 
from sklearn.metrics import mean_squared_error


X,y = make_moons(n_samples = 500, noise = 0.30, random_state = 42) # 데이터 셋 로드 

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

ada_clf = AdaBoostClassifier( # adaBoostClassifier 는 깊이가 3인 결정 트리를 기본 추정기로 한다.
    # 아래 코드에서 사용돠는 결정트리의 깊이는 1 이다. 즉 결정 노드 하나와 리프노드 두개로 이루어져있다.
    DecisionTreeClassifier(max_depth = 1), n_estimators = 200,
    algorithm = "SAMME.R", learning_rate = 0.5, random_state = 42)

ada_clf.fit(X_train, y_train)

# Gradient Boosting 
# 반복마다 샘플의 가중치를 수정하는 대신 이전 예측기가 만든 잔여 오차에 새로운 예측기를 학습시킨다.

np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:,0]**2 +0.05 * np.random.randn(100) # 뒤에 더해주는 부분은 noise

tree_reg1 = DecisionTreeRegressor(max_depth = 2, random_state = 42)
tree_reg1.fit(X,y)

y2 = y - tree_reg1.predict(X) # 첫번째 예측기에서 생긴 잔여 오차 
tree_reg2 = DecisionTreeRegressor(max_depth =2 , random_state =42)
tree_reg2.fit(X,y2) #오차로 두번째 모델 훈련 

y3 = y2 - tree_reg2.predict(X) # 세번째 오차 
tree_reg3 = DecisionTreeRegressor(max_depth =2 , random_state =42)
tree_reg3.fit(X,y3) # 오차로 세번째 훈련 

X_new = np.array([[0.4]])
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1,tree_reg2,tree_reg3))#세 개의 트리를 포합하는 앙상블 모델
#새로운 샘플에 대한 에측 ( 모든 트리의 예측을  더한 값)
print(y_pred)


#위와 동일한 기능을 가진 함수
gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators = 3, learning_rate = 1.0 , random_state =42)
gbrt.fit(X,y)

y_pred_gbrt = gbrt.predict(X_new)
print(y_pred_gbrt)

# 최적의 트리 수 찾기 위해 조기 종료 기법을 사용 할 수 있다.

X_train, X_val, y_train, y_val = train_test_split(X,y,random_state = 49)
gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators = 120, random_state =42)
gbrt.fit(X_train,y_train)

errors = [mean_squared_error(y_val, y_pred)
         for y_pred in gbrt.staged_predict(X_val)] # 조기 종료 기법

bst_n_estimators = np.argmin(errors) # 최적

gbrt_best = GradientBoostingRegressor(max_depth = 2, n_estimators = bst_n_estimators, random_state = 42)
# 최적의 모델로 
gbrt.fit(X_train,y_train)

print(np.argmin(errors))
print(np.min(errors))


gbrt = GradientBoostingRegressor(max_depth = 2, warm_start = True , random_state = 42)

min_val_error = float("inf")
error_going_up = 0
# 아래 코드는 연속해서 다섯번동안 검증 오차가 향상되지 않으면 훈련을 멈춘다.
for n_estimators in range(1,120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break # 조기 종료 

print(gbrt.n_estimators)# 61에 종료되었다는 거니까 55번째에 향상이 마지막 
print("최소 검증 MSE: " min_val_error)

[0.49995198]
[0.49995198]
55
0.002712853325235463
